In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
import time

In [2]:
NAME = f"Covid-Binary-Classifier-CNN-64x2-{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{NAME}")

In [3]:
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# only if you need to run multiple models at once so that it doesnt hog all the vram of the GPU

In [4]:
X = pickle.load(open("X.pickle", "rb"))
y = np.array(pickle.load(open("y.pickle", "rb")))

X = X/255.0

In [5]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1



In [6]:
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=5,
                      validation_split=0.3,
                      callbacks=[tensorboard])
        model.save("./cnn.model")

3-conv-64-nodes-0-dense-1647141308
Epoch 1/5
88/88 [==============================] - 11s 97ms/step - loss: 0.1931 - accuracy: 0.9119 - val_loss: 0.1474 - val_accuracy: 0.9393
Epoch 2/5
88/88 [==============================] - 8s 91ms/step - loss: 0.0616 - accuracy: 0.9757 - val_loss: 0.0433 - val_accuracy: 0.9867
Epoch 3/5
88/88 [==============================] - 8s 86ms/step - loss: 0.0539 - accuracy: 0.9829 - val_loss: 0.1139 - val_accuracy: 0.9576
Epoch 4/5
88/88 [==============================] - 9s 98ms/step - loss: 0.0746 - accuracy: 0.9725 - val_loss: 0.0647 - val_accuracy: 0.9800
Epoch 5/5
88/88 [==============================] - 9s 98ms/step - loss: 0.0465 - accuracy: 0.9832 - val_loss: 0.0700 - val_accuracy: 0.9825
INFO:tensorflow:Assets written to: ./cnn.model\assets


In [ ]:
model = Sequential()

model.add(Conv2D(64,(3,3), input_shape=X.shape[1:], activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # Converts 3D feature maps into 1D feature vectors
model.add(Dense(64, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X, y, batch_size=32, validation_split=0.1, epochs=30, callbacks=[tensorboard])
model.save("./cnn.model")

Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/conv2d_3/Conv2D (defined at <ipython-input-7-d17cedb2fd53>:16) ]] [Op:__inference_train_function_1808]

Function call stack:
train_function


In [ ]:
print(tf.__version__)

2.3.0
